## Schemas
Some data bases are using schemas (e.g. PostGres).
The schema information needs to be set before importing the churnmodels module - other the interface classes to the db will assume no schema

To set the schema name set the environment variable 
> os.environ["CHURN_DB_SCHEMA"]="some_schema"

Now the churnmodels module can be imported
> from churnmodels.schema import Subscription, Event, Account

... and the session variable can be instantiated


## Annoying way of importing the churnmodels Schema Tables
For data bases using schemas it is needed to get the schema information to python BEFORE we load in the Table classes.
The module "churnmodels.schema" uses the environment variable CHURN_DB_SCHEMA for the schema name that it handed over to a MetaData instance for SQLAlchemy.

In [2]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

options = {"user": "postgres",
           "pass": "password",
           "dbname": "churn",
           "schema": "biznet1"
           #"host" : "localhost" # ...if needed
           #"port" : "5432" # ...if needed
           }

# !!!! important part
# before even importing the churnmodels we need
os.environ["CHURN_DB_SCHEMA"]=options["schema"]
from churnmodels.schema import Subscription, Event, Account
# !!!! important part (end)

# connect to the database
from churnmodels.schema import get_db_uri
database_uri=get_db_uri(options, "postgres") # "postgres" names the dialect we are using
engine = create_engine(database_uri)
session = sessionmaker(bind=engine)()

# creating a simple statement
q=session.query(Account)
df=pd.read_sql_query(q.statement, engine)

print(q.statement)
print(df)

SELECT biznet1.account.id, biznet1.account.channel, biznet1.account.date_of_birth, biznet1.account.country 
FROM biznet1.account
          id    channel date_of_birth country
0          1  appstore2    1981-03-26      DE
1          2        web    1979-08-17      US
2          3  appstore1    1972-11-02      MX
3          4  appstore2    1976-02-08      GR
4          5  appstore2    1973-07-25      MX
...      ...        ...           ...     ...
14636  14637  appstore2    1972-08-11      CA
14637  14638  appstore2    1943-12-24      IN
14638  14639  appstore2    1952-10-26      KR
14639  14640  appstore2    1951-12-07      JP
14640  14641  appstore2    1948-05-04      US

[14641 rows x 4 columns]


## The comfortable way: importing a reflexive Schema
SQLAlchemy gives the opportunity to recovery the complete DDL information of a data base.
By reflection technique of python we can read in a module dynamically containing classes wrapping our DB tables.


In [4]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

from churnmodels.schema import get_schema_rfl, get_db_uri

options = {"user": "postgres",
           "pass": "password",
           "dbname": "churn",
           "schema": "biznet1"
           #"host" : "localhost" # ...if needed
           #"port" : "5432" # ...if needed
           }
# tables is a (dynamical) module containg Wrapper classes for our data base
tables=get_schema_rfl(options)

# connect to the database
db_uri=get_db_uri(options, "postgres") # "postgres" names the dialect we are using
engine=create_engine(db_uri)
session = sessionmaker(bind=engine)()

# creating a simple statement
q=session.query(tables.Account)
df=pd.read_sql_query(q.statement, engine)

print(q.statement)
print(df)


SELECT biznet1.account.id, biznet1.account.channel, biznet1.account.date_of_birth, biznet1.account.country 
FROM biznet1.account
          id    channel date_of_birth country
0          1  appstore2    1981-03-26      DE
1          2        web    1979-08-17      US
2          3  appstore1    1972-11-02      MX
3          4  appstore2    1976-02-08      GR
4          5  appstore2    1973-07-25      MX
...      ...        ...           ...     ...
14636  14637  appstore2    1972-08-11      CA
14637  14638  appstore2    1943-12-24      IN
14638  14639  appstore2    1952-10-26      KR
14639  14640  appstore2    1951-12-07      JP
14640  14641  appstore2    1948-05-04      US

[14641 rows x 4 columns]
